In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sharedtaskhatespeechincaste/dev.csv
/kaggle/input/sharedtaskhatespeechincaste/train (1).csv
/kaggle/input/sharedtaskhatespeechincaste/test (1).csv


# Module Import

In [2]:
import pandas as pd
import numpy as np
import re
import string
import torch
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from transformers import AdamW, AutoTokenizer
from sklearn.utils import class_weight
import itertools
from scipy.stats import mode
from torch.utils.data import Dataset

In [3]:
tamil_train = pd.read_csv('/kaggle/input/sharedtaskhatespeechincaste/train (1).csv')  # Replace with the correct path to your file
tamil_dev = pd.read_csv('/kaggle/input/sharedtaskhatespeechincaste/dev.csv')
tamil_test = pd.read_csv('/kaggle/input/sharedtaskhatespeechincaste/test (1).csv')


In [4]:
tamil_test.shape

(1576, 2)

In [5]:
tamil_test.head()

,id,text
0,4594,Humanity paatha intha slag countries prblm var...
1,3593,Avanga Holi kondada oruku poranga da
2,2533,Avangala avanga ooruke anupura vazhiya parunga.
3,6975,பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...
4,6011,Caste is there parayan think to marry higheru...


In [6]:
# Select the second and third columns (columns with index 1 and 2)
tamil_train = tamil_train.iloc[:, 1:3]  # Select columns from index 1 to 2
tamil_train = tamil_train.rename(columns={1: "text", 2: "label"})  # Rename columns to 'text' and 'label'
tamil_train['label'] = pd.Categorical(tamil_train['label'])  # Convert 'label' to categorical type

tamil_dev = tamil_dev.iloc[:, 1:3]  # Select columns from index 1 to 2
tamil_dev = tamil_dev.rename(columns={1: "text", 2: "label"})  # Rename columns to 'text' and 'label'
tamil_dev['label'] = pd.Categorical(tamil_dev['label'])  # Convert 'label' to categorical type


tamil_test = tamil_test.iloc[:, 1:2]  # Select columns from index 1 to 2
tamil_test = tamil_test.rename(columns={1: "text"})  # Rename columns to 'text' and 'label'


In [7]:
tamil_test.shape

(1576, 1)

In [8]:
tamil_test.head()

,text
0,Humanity paatha intha slag countries prblm var...
1,Avanga Holi kondada oruku poranga da
2,Avangala avanga ooruke anupura vazhiya parunga.
3,பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...
4,Caste is there parayan think to marry higheru...


# Preprocessing

lowercase & punctuation


In [9]:
# def lowercase_latin(text):
#     return re.sub(r'[A-Z]', lambda x: x.group(0).lower(), text)

# # Apply the function to the 'text' column
# tamil_train['text'] = tamil_train['text'].apply(lowercase_latin)
# tamil_dev['text'] = tamil_dev['text'].apply(lowercase_latin)
# tamil_test['text'] = tamil_test['text'].apply(lowercase_latin)

handling emojis

In [10]:
# import re
# import pandas as pd
# import emoji

# # Regular expression pattern to match emojis or emoticons
# emoji_pattern = re.compile(
#     "[\U0001F600-\U0001F64F"  # emoticons
#     "\U0001F300-\U0001F5FF"  # symbols & pictographs
#     "\U0001F680-\U0001F6FF"  # transport & map symbols
#     "\U0001F700-\U0001F77F"  # alchemical symbols
#     "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
#     "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
#     "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
#     "\U0001FA00-\U0001FA6F"  # Chess symbols
#     "\U0001FA70-\U0001FAFF"  # Symbols for Legacy Computing
#     "\U00002702-\U000027B0"  # Dingbats
#     "\U000024C2-\U0001F251"  # Enclosed characters
#     "]+"
# )

# # Function to extract emojis from the text
# def extract_emojis(text):
#     return emoji_pattern.findall(text)

# # Function to replace emojis with their text meaning
# def replace_emojis_with_meaning(text):
#     return emoji.demojize(text, delimiters=(" ", " "))  # Converts emojis to text with spaces

# # Function to get the rows containing emojis and also return the emojis in those rows
# def get_rows_with_emojis(text_series):
#     rows_with_emojis = text_series[text_series.apply(lambda x: bool(emoji_pattern.search(x)))]
#     emojis = rows_with_emojis.apply(extract_emojis)
#     return rows_with_emojis, emojis

# # train
# emoji_rows, emojis = get_rows_with_emojis(tamil_train['text'])
# print(f"Number of rows containing emojis or emoticons: {len(emoji_rows)}")
# tamil_train.loc[emoji_rows.index, 'text'] = tamil_train.loc[emoji_rows.index, 'text'].apply(replace_emojis_with_meaning)

# # dev
# emoji_rows, emojis = get_rows_with_emojis(tamil_dev['text'])
# print(f"Number of rows containing emojis or emoticons: {len(emoji_rows)}")
# tamil_dev.loc[emoji_rows.index, 'text'] = tamil_dev.loc[emoji_rows.index, 'text'].apply(replace_emojis_with_meaning)

# # for test
# emoji_rows, emojis = get_rows_with_emojis(tamil_test['text'])
# print(f"Number of rows containing emojis or emoticons: {len(emoji_rows)}")
# tamil_test.loc[emoji_rows.index, 'text'] = tamil_test.loc[emoji_rows.index, 'text'].apply(replace_emojis_with_meaning)


In [11]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek You (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A** Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A**",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A** Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don’t care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can’t stop laughing"
}


In [12]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [13]:
# def remove_punctuation(text):
#     return re.sub(f"[{string.punctuation}]", "", text)

# tamil_train['text'] = tamil_train['text'].apply(remove_punctuation)
# tamil_dev['text'] = tamil_dev['text'].apply(remove_punctuation)
# tamil_test['text'] = tamil_test['text'].apply(remove_punctuation)

In [14]:
tamil_train.head()

,text,label
0,Indha ariya kandupidippin moolam neenga solla ...,0
1,@vijayakumarp7959 unmai therincha nee pesu,0
2,Inga erukka yella dev... boys vadakkan vadakk...,1
3,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த 350 கோடியை ...,1
4,Mumbai Bangalore la 80% percentage outsiders,1


In [15]:
tamil_dev.head()

,text,label
0,"HOSUR also mini North India bro ,,",0
1,@rajaa9979 வணிகர் சங்கங்களின் தலைமை தமிழரிடத...,0
2,Already Telangana becomes Hindi belt 🥺,0
3,வட மாநிலத்தொழிளார் தமிழ்நாட்டினரரை தாக்கினால் ...,0
4,தமிழன் வட இந்தியாவிலும் கேரளா ஆந்திரா கர்ந...,0


# dataset & dataloader

In [16]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.device_count())

True
12.1
2


In [17]:
torch.cuda.set_device(1)
device = 'cuda'
device = device if torch.cuda.is_available() else 'cpu'

In [18]:
class tamil_Offensive_Dataset(Dataset):
    def __init__(self, encodings, labels=None, bpe=False):
        self.encodings = encodings
        self.labels = labels
        self.is_bpe_tokenized = bpe

    def __getitem__(self, idx):
        if not self.is_bpe_tokenized:
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        else:
            item = {
                'input_ids': torch.LongTensor(self.encodings[idx].ids),
                'attention_mask': torch.LongTensor(self.encodings[idx].attention_mask)
            }
        # Only add labels if available (for training/validation datasets)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels) if self.labels is not None else len(self.encodings)


# model list

In [ ]:
# Define models and configurations
models = ['TamilBert']
wbools = [True]

# Training Loop

In [20]:
num_of_epochs=1

In [21]:
# Track models and their predictions
all_predictions = {}

# Iterate over models
for model, wbool in list(itertools.product(models, wbools)):
    loss_weighted = wbool
    # model_predictions = []
    
    # Model selection and initialization based on the model type
    if model == 'MURIL':
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
        model_instance = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=2)  # Change to 2 for binary classification
        model_name = 'MURIL_base_cased_tamil'
    

    if model == 'XLMR':
        from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
        tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
        model_instance = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=2)  # Change to 2 for binary classification
        model_name = 'XLMroberta_large_tamil'
    
    
    if model == 'mbertbase':
        from transformers import BertTokenizer, BertForSequenceClassification
        tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
        model_instance = BertForSequenceClassification.from_pretrained('google-bert/bert-base-multilingual-cased', num_labels=2)  # Change to 2 for binary classification
        model_name = 'Mbert_base_cased_tamil'
    
    if model == 'indic':
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
        model_instance = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=2)  # Change to 2 for binary classification
        model_name = 'Indic_bert_tamil'

    if model == 'TamilBert':
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-bert")
        model_instance = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/tamil-bert", num_labels=2)  # Change to 2 for binary classification
        model_name = 'TamilBert'

    

    if loss_weighted:
        model_name = model_name + '_weighted'
    print("Model: {}".format(model_name))
    


    from transformers import AdamW
    optimizer = AdamW(model_instance.parameters(), lr=1e-5)

    train_batch_sentences = list(tamil_train['text'])  # Extract the text data
    train_batch_labels = list(tamil_train['label'])   # Extract the numeric labels (0 or 1)
    dev_batch_sentences = list(tamil_dev['text'])      # Extract the text data
    dev_batch_labels = list(tamil_dev['label'])
    
    test_batch_sentences = list(tamil_test['text'])      # Extract the text data
    
    print(f"Length of test_batch_sentences: {len(test_batch_sentences)}")
    print(test_batch_sentences[:5])  # Print first 5 sentences

    if 'parameters' in tokenizer.__dict__.keys() and tokenizer.__dict__['_parameters']['model'] == 'ByteLevelBPE':
        train_encodings = tokenizer.encode_batch(train_batch_sentences)
        dev_encodings = tokenizer.encode_batch(dev_batch_sentences)
        # test_encodings = tokenizer.encode_batch(test_batch_sentences)
    else:
        train_encodings = tokenizer(train_batch_sentences, padding='max_length', truncation=True, max_length=115, return_tensors="pt")
        dev_encodings = tokenizer(dev_batch_sentences, padding='max_length', truncation=True, max_length=115, return_tensors="pt")
        test_encodings = tokenizer(
            test_batch_sentences,  # Make sure this is a list of 1576 strings
            padding=True, 
            truncation=True, 
            max_length=115, 
            return_tensors="pt"
        )



    train_labels = torch.tensor(train_batch_labels)
    dev_labels = torch.tensor(dev_batch_labels)

    print(f"Length of test_encodings: {len(test_encodings)}")


    # Defining Datasets
    train_dataset = tamil_Offensive_Dataset(train_encodings, train_labels, bpe = False)
    dev_dataset = tamil_Offensive_Dataset(dev_encodings, dev_labels, bpe = False)
    test_dataset = tamil_Offensive_Dataset(test_encodings, bpe = False)
    

    # model_instance.to(device)
    # model_instance.to(device)

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model_instance.to(device)
    best_val_f1 = 0
    count = 0



    from sklearn.utils import class_weight
    import torch.nn as nn
    # weights = class_weight.compute_class_weight('balanced',np.unique(train_batch_labels),train_batch_labels)
    # weights = class_weight.compute_class_weight('balanced', np.unique(train_batch_labels), train_batch_labels)
    weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_batch_labels), y=train_batch_labels)

    # weights = np.exp(weights)/np.sum(np.exp(weights))
    class_weights = torch.FloatTensor(weights).to(device)
    loss_function = nn.CrossEntropyLoss(weight=class_weights, reduction='mean')

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    print(f"Total test samples: {len(test_dataset)}")
    print(f"Total test batches: {len(test_loader)}")



    for epoch in range(num_of_epochs):
        train_preds = []
        train_labels = []
        total_train_loss = 0
        model_instance.train()
        print("==========================================================")
        print("Epoch {}".format(epoch))
        print("Train")
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model_instance(input_ids, attention_mask=attention_mask, labels=labels)
            if loss_weighted:
                loss = loss_function(outputs.logits, labels)
            else:
                loss = outputs[0]
            loss.backward()
            optimizer.step()

            for logits in outputs[1].detach().cpu().numpy():
                train_preds.append(np.argmax(logits))
            for logits in labels.cpu().numpy():
                train_labels.append(logits)
            total_train_loss += loss.item()/len(train_loader)

        print("Dev")
        dev_preds = []
        model_instance.eval()
        total_val_loss = 0
        with torch.set_grad_enabled(False):
            for batch in tqdm(dev_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                # outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                outputs = model_instance(input_ids, attention_mask=attention_mask, labels=labels)

                if loss_weighted:
                    loss = loss_function(outputs[1], labels)
                else:
                    loss = outputs[0]
                total_val_loss += loss.item()/len(dev_loader)

                for logits in outputs[1].cpu().numpy():
                    dev_preds.append(np.argmax(logits))

        y_true = dev_batch_labels
        y_pred = dev_preds
        print(f"y_true shape: {len(y_true)}")
        print(f"y_pred shape: {len(y_pred)}")
        target_names = ['Not caste/immigration hate speech', 'Caste/immigration hate speech']
        train_report = classification_report(train_labels, train_preds, target_names=target_names)
        report = classification_report(y_true, y_pred, target_names=target_names)
        val_f1 = f1_score(y_true, y_pred, average='macro')

        if val_f1 > best_val_f1:
            test_preds = []  # Reset before loop
            test_sentences_processed = 0  # Track processed sentences
            
            with torch.no_grad():
                for i, batch in enumerate(tqdm(test_loader)):
                    try:
                        input_ids = batch['input_ids'].to(device)
                        attention_mask = batch['attention_mask'].to(device)
            
                        outputs = model_instance(input_ids, attention_mask=attention_mask)
                        batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            
                        test_preds.extend(batch_preds)
                        test_sentences_processed += len(batch_preds)
            
                        print(f"✅ Batch {i+1}: Processed {len(batch_preds)} samples")
            
                    except Exception as e:
                        print(f"❌ Error in batch {i+1}: {e}")
            
            print(f"⚠️ Total processed test sentences: {test_sentences_processed}")
            print(f"⚠️ Expected test sentences: {len(test_batch_sentences)}")
            print(f"⚠️ Final length of test_preds: {len(test_preds)}")


            test_df = pd.DataFrame({'text': test_batch_sentences, 'prediction': test_preds})
            csv_path = os.path.join(model_save_path, model_name + "_test_predictions.csv")
            test_df.to_csv(csv_path, index=False)

            best_val_f1 = val_f1
            count = 0

            # Store best predictions
            all_predictions[model_name] = y_pred.copy()
        else:
            count += 1

        print(train_report)
        print(report)
        print("Epoch {}, Train Loss = {}, Val Loss = {}, Val F1 = {}, Best Val f1 = {}, stagnant = {}".format(epoch, total_train_loss, total_val_loss, val_f1, best_val_f1, count))
        if count == 5:
            print("No increase for 5 epochs, Stopping ...")
            break

    import gc
    del model_instance
    gc.collect()
    torch.cuda.empty_cache()
    


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

...

100%|██████████| 50/50 [00:04<00:00, 10.88it/s]


y_true shape: 787
y_pred shape: 787


  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-18-360a0a568ef9>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 1/1 [00:00<00:00, 30.39it/s]


✅ Batch 1: Processed 3 samples
⚠️ Total processed test sentences: 3
⚠️ Expected test sentences: 1576
⚠️ Final length of test_preds: 3


ValueError: All arrays must be of the same length

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define model path (same as where you saved it)
model_path = "/kaggle/working/finetuned_models/MURIL_base_cased_tamil_weighted"

# Load model + tokenizer directly
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set to evaluation mode
model.eval()
print("Model loaded successfully!")


In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm  # For progress tracking
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_labels(model, tokenizer, texts, batch_size=16):
    model.eval()  # Ensure model is in evaluation mode
    predictions = []

    # Tokenize texts
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=115, return_tensors="pt")
    
    # Create a DataLoader
    dataset = torch.utils.data.TensorDataset(encodings["input_ids"], encodings["attention_mask"])
    dataloader = DataLoader(dataset, batch_size=batch_size)

    with torch.no_grad():  # Disable gradient calculations
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids, attention_mask = batch
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Get raw predictions
            
            batch_preds = torch.argmax(logits, dim=1).cpu().numpy()  # Convert to class labels
            predictions.extend(batch_preds)

    return np.array(predictions)

# Get texts from tamil_dev and tamil_test
tamil_dev_texts = list(tamil_dev['text'])  # Dev dataset
tamil_test_texts = list(tamil_test['text'])  # Test dataset

# Predict
dev_predictions = predict_labels(model, tokenizer, tamil_dev_texts)
test_predictions = predict_labels(model, tokenizer, tamil_test_texts)

# Print results
print(f"Dev Predictions: {dev_predictions[:10]}")  # Print first 10 predictions
print(f"Test Predictions: {test_predictions[:10]}")


# Result of every models

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
from tabulate import tabulate
from rich.console import Console
from rich.table import Table

console = Console()

for model, preds in all_predictions.items():
    console.print(f"\n[bold cyan]🔹 Best Predictions for {model}:[/bold cyan] {preds[:10]}", style="bold green")  

    # Compute metrics
    model_f1 = f1_score(y_true, preds, average='macro')
    model_accuracy = accuracy_score(y_true, preds)
    model_report = classification_report(y_true, preds, target_names=target_names, digits=4, output_dict=True)

    # Displaying results using Rich Table
    table = Table(title=f"[bold magenta]Performance Metrics for {model}[/bold magenta]", show_header=True, header_style="bold yellow")
    table.add_column("Metric", style="bold cyan")
    table.add_column("Score", justify="center", style="bold white")
    
    table.add_row("Accuracy", f"{model_accuracy:.4f}")
    table.add_row("F1 Score (Macro)", f"{model_f1:.4f}")

    console.print(table)  # Print the table

    # Displaying classification report in a tabular format
    headers = ["Class", "Precision", "Recall", "F1-score", "Support"]
    rows = []

    for label, metrics in model_report.items():
        if label in target_names:  # Avoid non-label keys
            rows.append([label, f"{metrics['precision']:.4f}", f"{metrics['recall']:.4f}", f"{metrics['f1-score']:.4f}", int(metrics['support'])])

    print("\nClassification Report:")
    print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))  # Fancy grid format
    print("=" * 100)  # Separator for better readability


# Majority voting and result

In [ ]:
# Convert predictions to a NumPy array
predictions_matrix = np.array(list(all_predictions.values()))  # Shape: (num_models, num_samples)

# Apply majority voting
ensemble_predictions, _ = mode(predictions_matrix, axis=0)

# Convert the result to a 1D array
ensemble_predictions = ensemble_predictions.flatten()

# Compute evaluation metrics
ensemble_f1 = f1_score(y_true, ensemble_predictions, average='macro')  # Macro F1-score
ensemble_accuracy = accuracy_score(y_true, ensemble_predictions)  # Accuracy score
ensemble_report = classification_report(y_true, ensemble_predictions, target_names=target_names)

# Print results
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
print(f"Ensemble F1 Score: {ensemble_f1:.4f}")
print("\nClassification Report:\n", ensemble_report)
